## **Student's Autograder**
This is a project to calculate quiz and term scores from q&a pdf and response pdf.

Whats left: 

* Editing pdf


### Importing the required libraries 

In [1]:
import os
import pandas as pd
import numpy as np
import tabula
from ipywidgets import widgets
from PyPDF2 import PdfFileReader, PdfFileWriter
from PyPDF2.generic import DecodedStreamObject, EncodedStreamObject,ArrayObject

In [2]:
#debugging for notebooks current directory for reading files later.
import os
os.getcwd()

'C:\\Users\\506761\\OneDrive - Tata Steel Limited\\Documents\\Python Scripts'

### Reading pdfs

In [3]:
resp_pdf = widgets.Text(
 placeholder='Enter filename with extension',
 description='Response pdf:'
 )
qa_pdf = widgets.Text(
 placeholder='Enter filename with extension',
 description='Q&A pdf:'
 )
#POD22S1C11500062.pdf
#IIT M DAD DIPLOMA AN2 EXAM QPD1.pdf
display(resp_pdf,qa_pdf)

Text(value='', description='Response pdf:', placeholder='Enter filename with extension')

Text(value='', description='Q&A pdf:', placeholder='Enter filename with extension')

In [85]:
filename=qa_pdf.value

### Variables initialization

In [86]:
my_question="Question ID"
my_answer="Options Selected"
unanswered="Unanswered"
file_qid="Question Id"

Students response for exam dataframe

In [87]:
rpdf_df=PdfFileReader(resp_pdf.value)

In [88]:
#Updated response pdf reading method from tabula to raw 
myans=[]
for i in range(rpdf_df.getNumPages()):
    myans+=(rpdf_df.pages[i].extractText().split("\n"))
myans=myans[myans.index("Question ID"):]
myans=pd.DataFrame(np.vstack((myans[2::2],myans[3::2])).T,columns=[myans[0],myans[1]])
myans["Question ID"]=myans["Question ID"].apply(int)
myans

,Question ID,Options Selected
0,640653445429,Unanswered
1,640653445430,Unanswered
2,640653445431,Unanswered
3,640653445432,Unanswered
4,640653445433,Unanswered
...,...,...
272,640653445726,Unanswered
273,640653445727,Unanswered
274,640653445730,Unanswered
275,640653445731,Unanswered


In [89]:
# with pd.option_context("display.max_rows",None):
#     print(df3)

### Useful functions

In [90]:
def gentextarr(content):
    """
    Returns an array of text block properties of pdf
    Parameter:
        content: string object of page content obtained by page.getContents().getData().decode() 
    """
    lines = content.splitlines()
    result = []
    in_text = False
    for line in lines:
        if line == "BT":
            in_text = True
            temp=[]

        elif line == "ET":
            in_text = False
            result.append(temp)
            temp=[]

        elif in_text:
            if line[-2:].lower()=="tj" or line[-2:].lower()=="tf" or line[-2:].lower()=="rg":
                temp.append(line[:])
    return result

def gen_cmap(font,pg):
    """
    Returns a character map dictionary of a pdf 'filename' page 'pg' and font 
    Parameter:
     font: font key name in meta properties of pdf.
     pg: int page number of pdf
    """
    # pdf = PdfFileReader(filename)
    # page = pdf.getPage(pg)
    page=pg
    temp=False
    cmap={}
    for i in (page.getObject()['/Resources']['/Font'][font]['/ToUnicode'].getObject().getData().decode().split('\n')):
        if "endbfrange" in i:
            temp=False
        if temp:
            cmap[i[1:5]]=i[-5:-1]
        if "beginbfrange" in i:
            temp=True
    return cmap

def font_decoder(text,cmap):
    """
    Returns decoded word from pdf text hex code. 
    """

    count=0
    word=""
    temp=""
    for i in text:
        count+=1
        if count==4:
            temp+=i
            word+=chr(int(cmap.get(temp.lower(),temp.lower()),16))
            temp=""
            count=0
        else:
            temp+=i
    return word


## PDF Content

In [91]:
"""
This piece of code extracts question answer pdf text content as well as correct answer key.
"""
pdf = PdfFileReader(filename)
sum=0
df=[[file_qid,"Correct Marks","Answers"]]
m_bool=False
marks=""
ans_bool=False
ans=""
row=[]
temp=[]
s_head=[]
temp_head=""
ishead=False
for p in range(0,pdf.getNumPages()):
    content=pdf.pages[p].getContents()
    #a different type content
    if type(content)==ArrayObject:
        m=""
        for n in content:
            m+=n.getObject().getData().decode()
        content=m
    elif type(content)==EncodedStreamObject or type(content)==DecodedStreamObject:
        content=content.getData().decode()
    for i in gentextarr(content):
        text=""
        ans_bool=False
        for j in i:
            # getting character map for a font type
            if j[-2:].lower()=="tf":
                cmap=gen_cmap(j.split()[0],pdf.pages[p])
                #checking for subject headings with font size =18
                if j.split()[1]=="18":
                    ishead=True
            # checking green color as ind of correctness        
            if j=="0 0.50196 0 rg" and len(row)>1:
                ans_bool=True
    
            elif j[-2:].lower()=="tj":
                text=font_decoder(j[1:-3],cmap)
                if ishead:
                    ishead=False
                    temp_head=text
                
                if (not m_bool) and (not ans_bool):
                    # checking for question id
                    if str(file_qid) in text:
                        if len(temp)>0:
                            row.append(temp)
                            df.append(row)
                        row=[]
                        temp=[]
                        if text.split()[7].isnumeric():
                            s_head.append(temp_head)
                            row=[text.split()[7]]
                            m_bool=True
                    
                # extracting marks
                elif m_bool:
                    if "Correct Marks" in text:
                        # appending to heading list if marks is 0
                        # if text.split()[-1]=="0":
                        #     s_head.append(temp_head)
                        if len(row)==1:
                            row.append(text.split()[3])
                        m_bool=False
                        
                #extracting answer key
                elif ans_bool:
                    if text!=" ":
                        if len(text)>=13 and text[:13].isnumeric():
                            temp.append(text[:13])
                        elif len(text.split())==3 and "to" in text.split()[1]:
                            t=text.split()
                            temp=[float(t[0]),float(t[-1])]
                        elif len(temp)==0:
                            temp.append(text)
if len(temp)>=1:
    # s_head.append(temp_head)
    row.append(temp)
    df.append(row)

In [92]:
print(len(df))
df[1]

278


['640653445429', '0.5', ['6406531484269']]

In [93]:
df2=pd.DataFrame(df[1:],columns=df[0])
df2[file_qid]=df2[file_qid].apply(int)
df2["Subjects"]=np.array(s_head[-df2.shape[0]:])
df2=df2.sort_values([file_qid])
df2.tail()

,Question Id,Correct Marks,Answers,Subjects
273,640653445727,8,[6406531485133],System commands
270,640653445728,8,"[6406531485135, 6406531485137]",System commands
274,640653445730,6,"[6406531485140, 6406531485142]",System commands
275,640653445731,6,[6406531485148],System commands
276,640653445732,6,[6406531485151],System commands


In [94]:
def evaluate(df2):
    
    #Multi option
    df2['Score'] = df2.apply(lambda x: (len(set(x["My answer"].split(","))\
                                            .intersection(set(x["Answers"])))/len(x["Answers"]))*float(x['Correct Marks'])\
                             if type(x["Answers"][0])==str and len(x["Answers"][0])==13 and\
                                len(set(x["My answer"].split(",")))<=len(set(x["Answers"])) and\
                                set(x["My answer"].split(",")).issubset(set(x["Answers"]))\
                             else x["Score"], axis=1)
    #integer ranged
    df2['Score'] = df2.apply(lambda x: float(x['Correct Marks'])\
                             if len(x["My answer"])<13 and len(x["My answer"])>0 and\
                                len(x['Answers'])>=1 and\
                                float(x['My answer']) >= float(x["Answers"][0]) and\
                                float(x['My answer']) <= float(x["Answers"][-1])\
                             else x["Score"], axis=1)
    return df2

## Final score

### Method 1: using index of 0 correct marks

In [95]:
# s_index=df3[df3["Correct Marks"]=="0"].index
# s_index

In [96]:
# subjects={}
# for i in range(len(s_index)):
#     if i==len(s_index)-1:
#         subjects[df3["Subjects"][s_index[i]]]=df3.iloc[s_index[i]:]
#     else:
#         subjects[df3["Subjects"][s_index[i]]]=df3.iloc[s_index[i]:s_index[i+1]]
# subjects.keys()

In [97]:
# reports=[]
# for i in subjects.values():
#     print(i["Correct Marks"].apply(float).values)
#     reports.append([i["Subjects"].unique()[0],i["Correct Marks"].apply(float).values.sum(),i["Score"].sum()])

In [98]:
# for i in reports:
#     print(i[0],":",i[2]/i[1]*100,"%")

### Method 2: Using groupby function

In [99]:
df3=df2.set_index('Question Id').combine_first(myans.set_index('Question ID')).reset_index().replace(["Unanswered",np.NAN],"")

In [100]:
df3.columns=["Question Id","Answers","Correct Marks","My answer","Subjects" ]
df3["Score"]=np.zeros(df3.shape[0])
df3=evaluate(df3)

In [101]:
df3[df3["Subjects"]=="MLT"]

,Question Id,Answers,Correct Marks,My answer,Subjects,Score
183,640653445629,[6406531484861],0,6406531484861,MLT,0.0
184,640653445630,[6406531484865],7,6406531484865,MLT,7.0
185,640653445631,[6406531484870],6,6406531484870,MLT,6.0
186,640653445632,[6406531484871],6,6406531484871,MLT,6.0
187,640653445633,[6406531484876],6,6406531484873,MLT,0.0
188,640653445634,[6406531484881],6,6406531484881,MLT,6.0
189,640653445635,[6406531484882],4,6406531484882,MLT,4.0
190,640653445636,[6406531484888],6,6406531484888,MLT,6.0
191,640653445637,[6406531484889],5,6406531484889,MLT,5.0
192,640653445638,"[6406531484891, 6406531484892]",5,"6406531484891,6406531484892",MLT,5.0


In [102]:
df3["Correct Marks"]=df3["Correct Marks"].apply(float)

In [103]:
report2=pd.DataFrame(df3.groupby("Subjects")[["Correct Marks","Score"]].sum())

In [104]:
report2["Percentage"]=(report2["Score"]*100/report2["Correct Marks"]).round(2)

In [105]:
report2.columns=["Total Marks","Scored","Percentage"]
headers = {
    'selector': 'thead',
    'props': 'background-color: #0dcaf0; color: white;'
}
report2.style.applymap(lambda v: 'color:red;' if (v == 0) else 'color:green',subset=["Percentage","Scored"])\
    .set_table_styles([headers])\
    .format({"Total Marks":'{:.2f}',"Scored":'{:.2f}',"Percentage":'{0:,.2f}%'})

,Total Marks,Scored,Percentage
Subjects,,,
AppDev1,50.00,0.00,0.00%
AppDev2,50.00,0.00,0.00%
BDM,16.00,0.00,0.00%
Business Analytics,20.00,0.00,0.00%
CT,50.00,0.00,0.00%
DBMS,50.00,0.00,0.00%
Java,50.00,0.00,0.00%
MLF,50.00,0.00,0.00%
MLP,50.00,0.00,0.00%


In [106]:
report2[report2["Percentage"]>0]

,Total Marks,Scored,Percentage
Subjects,,,
MLT,100.0,94.0,94.0


### Exporting dataframe as csv

In [107]:
df3[df3["Subjects"].isin(report2[report2["Percentage"]>0].index)].to_csv("summary.csv",index=False)
report2[report2["Percentage"]>0].to_csv("report_card.csv")

279

In [109]:
df2.shape

(277, 4)

In [110]:
df2[-df2.shape[0]:].shape

(277, 4)